## 0. initialization

In [ ]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import calendar
import datetime
from datetime import datetime, timedelta, date
import warnings
import os

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#settings
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 1200)
pd.set_option('display.width', 1200)
warnings.filterwarnings("ignore")
%matplotlib inline

#variables
colour=['maroon','r','g','darkgreen','c','teal','b','navy','indigo','m','deeppink','orange','sienna','yellow','khaki','olive','tan','black','grey','brown']

print("Packages installed")

In [ ]:
file_path = '/work/17_02_2024_cleaned_Data.csv'
df = pd.read_csv(file_path, sep=",", encoding="UTF-8")
file_path = '/work/RawData/clubs.csv'
clubs_df = pd.read_csv(file_path, sep=",", encoding="UTF-8")
#df.drop(columns=['player_club_id.1'], inplace=True)
df.rename(columns={'player_club_id': 'club_id'}, inplace=True)
#clubs_df.rename(columns={'name': 'club_name'}, inplace=True)
df = pd.merge(df, clubs_df[['club_id', 'name']], on='club_id', how='left')
print(df)
print(clubs_df)

# 1. Sum club total values and plot

In [ ]:
# Assuming your DataFrame is named df and the column with the player evaluations is named 'market_value_in_eur'
# Assuming the column with the club IDs is named 'club_id'
# Assuming the DataFrame with club names is named clubs_df and the column with the club names is named 'club_name'

# Group the data by 'club_id' and sum the player evaluations for each club
club_values = df.groupby(['club_id', 'year']).agg({"market_value_in_eur" : "sum"}).reset_index()

club_values.rename(columns = {"market_value_in_eur" : "club_value"}, inplace = True)

# Sort the clubs by total value in descending order
club_values_sorted = club_values.sort_values("club_value", ascending=False)

# Get the club names from clubs_df
club_names = clubs_df.set_index('club_id')['name']




In [ ]:
# Plot the data in a bar diagram
plt.figure(figsize=(10, 6))
club_values_sorted.plot(kind='bar')
plt.title('Total Player Value by Club')
plt.xlabel('Club ID')
plt.ylabel('Total Player Value')
plt.xticks(rotation=45)

# Set the labels of the bars to the club names
plt.xticks(ticks=range(len(club_values_sorted)), labels=club_names[club_values_sorted.index], rotation=45)

plt.show()

# Plot the top 10 clubs in a bar diagram
plt.figure(figsize=(10, 6))
club_values_sorted.head(10).plot(kind='bar')
plt.title('Top 10 Clubs by Total Player Value')
plt.xlabel('Club ID')
plt.ylabel('Total Player Value')
plt.xticks(rotation=45)

# Set the labels of the bars to the club names
plt.xticks(ticks=range(len(club_values_sorted.head(10))), labels=club_names[club_values_sorted.head(10).index], rotation=45)

plt.show()

# Plot the bottom 10 clubs in a bar diagram
plt.figure(figsize=(10, 6))
club_values_sorted.tail(10).plot(kind='bar')
plt.title('Bottom 10 Clubs by Total Player Value')
plt.xlabel('Club ID')
plt.ylabel('Total Player Value')
plt.xticks(rotation=45)

# Set the labels of the bars to the club names
plt.xticks(ticks=range(len(club_values_sorted.tail(10))), labels=club_names[club_values_sorted.tail(10).index], rotation=45)

plt.show()

In [ ]:
file_path = '/work/cleaned_data_27.csv'
df = pd.read_csv(file_path, sep=",", encoding="UTF-8")
# Density Plot of the Prices with a logaritmic transformation on our target value
plt.figure(figsize=(15, 8))

df['log_market_value_base10'] = np.log10(df['market_value_in_eur'])
sns.histplot(df['log_market_value_base10'], kde=True)
plt.xlabel("Price (M)", fontsize=12)
plt.ylabel('Density',fontsize=12)
plt.title("Density Plot of the Player Values", fontsize=16)
plt.show()

In [ ]:
from scipy.stats import norm 
import statistics 
df.sort_values(by='market_value_in_eur', inplace=True)

# If you want to reset the index after sorting
df.reset_index(drop=True, inplace=True)
df['log_market_value_base10'] = np.log(df['market_value_in_eur'])


x_axis = df['log_market_value_base10']

mean = statistics.mean(x_axis) 
sd = statistics.stdev(x_axis) 
  
plt.plot(x_axis, norm.pdf(x_axis, mean, sd)) 
plt.show() 

## Data distrubution

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


file_path = '/work/encoded_data_03_12.csv'
df = pd.read_csv(file_path, sep=",", encoding="UTF-8")


# Assuming your DataFrame is named 'df'
# Replace 'df' with your actual DataFrame name



# Set the number of columns for subplots
num_cols = len(df.columns)

# Calculate the number of rows needed based on the number of columns
num_rows = (num_cols + 1) // 2

# Create subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(12, 2.5 * num_rows))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate through each column and plot a histogram
for i, column in enumerate(df.columns):
    sns.histplot(df[column], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution of {column}')

# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


file_path = '/work/cleaned_data_03_12.csv'
df = pd.read_csv(file_path, sep=",", encoding="UTF-8")

# Assuming df_encoded is your preprocessed DataFrame containing player values and league information

# Get the unique leagues in the dataset
unique_leagues = df_encoded['league'].unique()

# Create a dictionary to store the number of players for each league
league_player_count = {}

# Iterate over each unique league
for league in unique_leagues:
    # Count the number of rows (players) belonging to the current league
    player_count = df_encoded[df_encoded['league'] == league].shape[0]
    
    # Store the player count for the current league in the dictionary
    league_player_count[league] = player_count

# Print the number of players for each league
for league, player_count in league_player_count.items():
    print(f"Number of players in {league}: {player_count}")



In [ ]:
import pandas as pd
file_path = '/work/cleaned_data_03_12.csv'
df = pd.read_csv(file_path, sep=",", encoding="UTF-8")

# Initialize a dictionary to store the sum of unique values for each column
unique_sum = {}

# Iterate through each column and calculate the sum of unique values
for column in df.columns:
    unique_values = df[column].nunique()
    unique_sum[column] = unique_values

# Print the sum of unique values for each column
for column, total in unique_sum.items():
    print(f"Sum of unique values in column '{column}': {total}")


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df contains your dataset
# If not, replace df with your actual DataFrame
df = pd.read_csv('../DataSets/EncodedData/OneHotEncodedData2.csv')
df = df.drop(["Name", "Market Value"], axis=1)

# Drop columns starting with "POS_"
pos_columns = [col for col in df.columns if col.startswith("POS_")]
df = df.drop(pos_columns, axis=1)

desired_columns_order = ['Height', 'Year', 'Club Goals', 'Club Goals Conceded', 'Goals', 'Assists', 
                         'Red Cards', 'Yellow Cards', 'Minutes Played', 'Age', 'Goals/Game', 'Assists/Game', 
                         'Goal Difference', 'Country Value', 'Club Value', 'League Value', 'Position Value', 
                         'Logarithmic Market Value']

# Reorder the columns
df = df[desired_columns_order]

# Compute the correlation matrix
corr = df.corr()
colors = sns.color_palette("coolwarm", as_cmap=True)
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
plt.figure(figsize=(12, 8))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


plt.title('Correlation Heatmap', fontsize=18)  # Adjust title font size
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})



# Draw the heatmap with the mask and correct aspect ratio
#sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, fmt=".2f", annot_kws={"size": 12})


plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter


# Read the DataFrame
df = pd.read_csv('../DataSets/EncodedData/OneHotEncodedData2.csv')

# Drop POS_ columns
pos_columns = [col for col in df.columns if col.startswith("POS_")]
df = df.drop(pos_columns, axis=1)

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

# Create a figure with two subplots
fig, axs = plt.subplots(2, figsize=(15, 12), gridspec_kw={'height_ratios': [4, 1]})

# Plot the main density plot
sns.histplot(df['Market Value'], kde=True, ax=axs[0])
axs[0].set_xlabel("Price (M)", fontsize=12)
axs[0].set_ylabel('Density', fontsize=12)
axs[0].set_title("Density Plot of the Player Values", fontsize=16)
axs[0].set_xlim(left=0)  # Set x-axis lower limit to 0
def millions_formatter(x, pos):
    return f'{x / 1000000:.0f}M'

formatter = FuncFormatter(millions_formatter)
axs[0].xaxis.set_major_formatter(formatter)

# Plot the zoomed-in version in the upper right corner
sns.histplot(df['Market Value'], kde=True, ax=axs[1])
axs[1].set_xlabel("Price (M)", fontsize=10)
axs[1].set_ylabel('Density', fontsize=10)
axs[1].set_xlim(left=0, right=30_000_000)  # Set x-axis limits from 0 to 30 million
axs[1].xaxis.set_major_formatter(formatter)

# Adjust layout to make room for the second plot
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:
# Create the main plot
import numpy as np
plt.figure(figsize=(15, 5))
ax = plt.gca()

# Plot the main density plot
sns.histplot(df['Market Value'], kde=True, ax=ax)
ax.set_xlabel("Price (M)", fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title("Density Plot of the Player Values", fontsize=16)
ax.set_xlim(left=0)  # Set x-axis lower limit to 0
def millions_formatter(x, pos):
    return f'{x / 1000000:.0f}M'
ax.tick_params(axis='both', which='major', labelsize=12)
formatter = FuncFormatter(millions_formatter)
ax.xaxis.set_major_formatter(formatter)

# Create a second plot overlaid on the main plot in the upper right corner
ax_zoom = plt.axes([0.3, 0.3, 0.5, 0.5])  # [left, bottom, width, height]
sns.histplot(df['Market Value'], kde=True, ax=ax_zoom)
ax_zoom.set_xlabel("Price (M)", fontsize=12)
ax_zoom.set_ylabel('Density', fontsize=12)
ax_zoom.set_xlim(left=0, right=20_000_000)  # Set x-axis limits from 0 to 30 million
ax_zoom.xaxis.set_major_formatter(formatter)
ax_zoom.tick_params(axis='both', which='major', labelsize=12)
mean_value = np.mean(df['Market Value'])
ax_zoom.axvline(mean_value, color='gray', linestyle='--', label='Mean Player Value')
ax_zoom.legend()
# Show the plot
plt.show()